In [1]:
import numpy as np

In [2]:
# Given Parameters
transmission_rate = [500, 1000, 1500, 2000, 2500, 3000]  # in kb/s
num_devices_list = [50, 100, 150, 200, 250, 300]
task_data = [5, 10, 15, 20, 25, 30, 35, 40, 45]  # in Mbit
max_delay = 0.5  # in seconds
device_computing_capacity = [0.5, 1]  # in GHz
num_of_mecs = 10
mec_computing_capacity = 8  # in GHz
transmission_power = 4  # in Watts

# Constants
H_m = 1e-96  # Consumption factor of electricity
y_nt = 1e-6  # Weighted factor of local time cost
y_ne = 1e-12  # Weighted factor of local energy consumption
e_m = 1e-7  # Energy required to calculate a single bit of task data for MEC server
noise_power_spectral_density = 1e-9  # in Watts/Hz
channel_gain = 2e-10  # 
path_loss_index = 4  # Typical urban area path loss exponent
bandwidth = 1e9  # 1 MHz bandwidth
max_energy = 1000  # in Joules
clock_period_range = (800, 1200)  # cycles per bit
penalty_factor = 10 ** -2.5

In [3]:
def fitness_function(x, task_data, transmission_power, bandwidth, noise_power_spectral_density, device_computing_capacity, mec_computing_capacity, apply_penalty=True):
    clock_period = np.mean(clock_period_range)  # Average clock period for simplicity

    # Initialize variables to avoid UnboundLocalError
    transmission_time = 0
    mec_task_execution_time = 0

    if x == 0:  # Local computation
        local_time_cost = task_data / (device_computing_capacity * 1e9)  # Converting GHz to Hz
        local_time_cost *= clock_period  # Adjust for clock period per bit
        local_energy_cost = H_m * (device_computing_capacity**2) * task_data
        total_energy_cost = local_energy_cost
    else:  # Edge server computation
        transmission_time = task_data / (bandwidth * np.log2(1 + (transmission_power * channel_gain) / (noise_power_spectral_density * bandwidth)))
        mec_task_execution_time = task_data / (mec_computing_capacity * 1e9) * clock_period  # Converting GHz to Hz and adjusting for clock period
        mec_energy_cost = transmission_power * transmission_time + e_m * task_data
        total_energy_cost = mec_energy_cost

    # Directly integrate energy and time cost into total cost
    total_cost = y_nt * (transmission_time + mec_task_execution_time) + y_ne * total_energy_cost

    # Apply a severe penalty if the energy cost exceeds the maximum allowed and apply_penalty is True
    if total_energy_cost > max_energy and apply_penalty:
        total_cost += (total_energy_cost - max_energy) * penalty_factor  # Adjust the penalty factor as necessary

    return total_cost


In [4]:
from concurrent.futures import ThreadPoolExecutor, as_completed

class QPSO:
    def __init__(self, num_devices, max_iter, task_data):
        self.num_devices = num_devices
        self.max_iter = max_iter
        self.task_data = np.array(task_data) * 1e6  # Convert Mbit to bits for task sizes
        self.X = np.random.randint(2, size=(num_devices, max_iter))  # Initial binary decisions for each device
        
        # Initialize Personal best decisions and their fitness
        self.P = np.zeros((num_devices, 1), dtype=int)
        self.P_fitness = np.full(num_devices, np.inf)

        # Initialize global best decision and its fitness
        self.g = np.zeros(num_devices, dtype=int)  # Placeholder, will be updated in evaluate_initial_decisions_parallel
        self.g_fitness = np.inf  # Important: initialize g_fitness before evaluate_initial_decisions_parallel

        # Parallel evaluation of initial decisions to find personal bests
        self.evaluate_initial_decisions_parallel()

        # Initialize global best decision set
        self.g = self.X[:, 0]
        self.g_fitness = np.min(self.P_fitness)  # Use the best of the initial personal bests
        g_index = np.argmin(self.P_fitness)
        self.g = self.X[:, g_index]

    def evaluate_fitness_parallel(self, tasks):
        with ThreadPoolExecutor() as executor:
            
            # Create a mapping of future to (i, t) pair
            future_to_it = {}
            
            for i, (task_size, device_capacity, t) in enumerate(tasks):
                future = executor.submit(fitness_function, self.X[i, t], task_size, transmission_power,
                                        bandwidth, noise_power_spectral_density, device_capacity, mec_computing_capacity)
                future_to_it[future] = (i, t)

            for future in as_completed(future_to_it):
                i, t = future_to_it[future]
                current_fitness = future.result()

                # Update personal and potentially global best
                if current_fitness < self.P_fitness[i]:
                    self.P[i, 0] = self.X[i, t]
                    self.P_fitness[i] = current_fitness
                    if current_fitness < self.g_fitness:
                        self.g = self.X[:, t]
                        self.g_fitness = current_fitness


    def evaluate_initial_decisions_parallel(self):
        tasks = [(self.task_data[i % len(self.task_data)], device_computing_capacity[i % len(device_computing_capacity)], 0) for i in range(self.num_devices)]
        self.evaluate_fitness_parallel(tasks)

    def run(self):
        for t in range(self.max_iter):
            tasks = [(self.task_data[i % len(self.task_data)], device_computing_capacity[i % len(device_computing_capacity)], t) for i in range(self.num_devices)]
            self.evaluate_fitness_parallel(tasks)

            # Quantum-inspired position updates for next iteration
            for i in range(self.num_devices):
                if t < self.max_iter - 1:
                    phi = np.random.uniform(0, 1)
                    self.X[i, t+1] = np.round(phi * self.P[i, 0] + (1 - phi) * self.g[i]).astype(int)
        return self.g

In [5]:
# Collect the best offloading strategies for each number of devices
best_offloading_strategies = {}
for num_devices in num_devices_list:
    qpso = QPSO(num_devices, max_iter=100, task_data=task_data)
    best_offloading_strategies[num_devices] = qpso.run()

In [6]:
#print decisions
best_offloading_strategies

{50: array([0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0,
        0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1,
        1, 1, 1, 0, 0, 1]),
 100: array([1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1,
        1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0,
        0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0,
        1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1,
        1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0]),
 150: array([0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1,
        0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1,
        0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0,
        0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0,
        1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0,
        1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1,
        0, 1, 1, 1, 1, 1

In [7]:
def compute_completion_time(x, task_data, bandwidth, noise_power_spectral_density, transmission_power, device_computing_capacity, mec_computing_capacity):
    """Compute the completion time for task data based on offloading decision."""
    clock_period = np.mean(clock_period_range)
    if x == 0:  # Local computation
        local_time_cost = task_data / (device_computing_capacity * 1e9)
        local_time_cost *= clock_period
        completion_time = local_time_cost
    else:  # Edge server computation
        transmission_time = task_data / (bandwidth * np.log2(1 + (transmission_power * channel_gain) / (noise_power_spectral_density * bandwidth)))
        mec_task_execution_time = task_data / (mec_computing_capacity * 1e9) * clock_period
        completion_time = transmission_time + mec_task_execution_time
    return completion_time

# Calculate the average completion time for each task_data size, assuming uniform offloading decisions across devices
avg_completion_times = []
for t_data in task_data:
    task_size_bits = t_data #* 1e6  # Convert Mbit to bits
    completion_times = []
    for offloading_decision in best_offloading_strategies[50]:
        device_capacity = device_computing_capacity[offloading_decision % len(device_computing_capacity)]
        c_time = fitness_function(offloading_decision, task_size_bits, bandwidth, noise_power_spectral_density, transmission_power, device_capacity, mec_computing_capacity)
        completion_times.append(np.mean(c_time))
    avg_completion_times.append(np.mean(completion_times))

avg_completion_times


[309112916734323.6,
 618225833468648.6,
 927338750202973.6,
 1236451666937298.8,
 1545564583671624.2,
 1854677500405949.5,
 2163790417140274.2,
 2472903333874599.5,
 2782016250608925.0]

In [8]:
# Calculate the average completion time for each task_data size, assuming uniform offloading decisions across devices
avg_completion_times = []
for d_list in num_devices_list:
    #task_size_bits = t_data * 1e6  # Convert Mbit to bits
    dev = d_list 
    completion_times = []
    for offloading_decision in best_offloading_strategies[300]:
        device_capacity = device_computing_capacity[offloading_decision % len(device_computing_capacity)]
        c_time = compute_completion_time(offloading_decision, dev, bandwidth, noise_power_spectral_density, transmission_power, device_capacity, mec_computing_capacity)
        completion_times.append(c_time)
    avg_completion_times.append(np.mean(completion_times))

avg_completion_times

[15.740281866660206,
 31.480563733320412,
 47.22084559998061,
 62.961127466640825,
 78.70140933330103,
 94.44169119996123]

In [9]:
# Calculate the average completion time for each task_data size, assuming uniform offloading decisions across devices
avg_completion_times = []
for rate in transmission_rate:
    #task_size_bits = t_data * 1e6  # Convert Mbit to bits
    tr = rate 
    completion_times = []
    for offloading_decision in best_offloading_strategies[50]:
        device_capacity = device_computing_capacity[offloading_decision % len(device_computing_capacity)]
        c_time = compute_completion_time(offloading_decision, tr, bandwidth, noise_power_spectral_density, transmission_power, device_capacity, mec_computing_capacity)
        completion_times.append(c_time)
    avg_completion_times.append(np.mean(completion_times))

avg_completion_times

[216.6090073393606,
 433.2180146787212,
 649.827022018082,
 866.4360293574424,
 1083.045036696803,
 1299.654044036164]

In [10]:
def compute_energy_consumption(x, task_data, transmission_power, device_computing_capacity):
    """Compute the energy consumption for task data based on offloading decision."""
    if x == 0:  # Local computation
        local_energy_cost = H_m * (device_computing_capacity**2) * task_data
        energy_consumption = local_energy_cost
    else:  # Edge server computation
        transmission_time = task_data / (bandwidth * np.log2(1 + (transmission_power * channel_gain) / (noise_power_spectral_density * bandwidth)))
        mec_energy_cost = transmission_power * transmission_time + e_m * task_data
        energy_consumption = mec_energy_cost
    return energy_consumption

# Calculate the average energy consumption for each task_data size, assuming uniform offloading decisions across devices
avg_energy_consumptions = []
for t_data in task_data:
    task_size_bits = t_data * 1e1# Convert Mbit to bits
    energy_consumptions = []
    for offloading_decision in best_offloading_strategies[50]:  
        device_capacity = device_computing_capacity[offloading_decision % len(device_computing_capacity)]  # Cycle through device capacities
        e_consumption = compute_energy_consumption(offloading_decision, task_size_bits, transmission_power, device_capacity)
        energy_consumptions.append(e_consumption)
    avg_energy_consumptions.append(np.mean(energy_consumptions))

avg_energy_consumptions


[86.64339293574423,
 173.28678587148846,
 259.9301788072327,
 346.5735717429769,
 433.2169646787212,
 519.8603576144654,
 606.5037505502097,
 693.1471434859538,
 779.7905364216982]